# Load the user connection data and connect to the SAP HANA database instance

Before running the next cell make sure [../0x00-setup/temp_user.ini](../0x00-setup/temp_user.ini) is copied to [../0x00-setup/user.ini](../0x00-setup/user.ini) and the user + password are set appropriately.

In [ ]:
from hana_ml.algorithms.pal.utility import Settings
myhost, myport, myuser, mypwd = Settings.load_config("../0x00-setup/user.ini")

In [ ]:
from hana_ml import dataframe as hdf
myconn=hdf.ConnectionContext(
    address=myhost, 
    port=myport, 
    user=myuser,
    password=mypwd
)
print(f"Connected to SAP HANA db version {myconn.hana_version()} \nat {myhost}:{myport} as user {myuser}")

In [ ]:
print(myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

# Tables from SAP HANA

In [ ]:
hdf_titanic_train=myconn.table('TRAIN', schema='TITANIC')

# Random Decision Tree classification

Experiment 2: Split into Train and Test for fit

In [ ]:
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification

In [ ]:
uc_rdt_v2 = UnifiedClassification(func='RandomDecisionTree')

🤓 **Let's discuss**:
- Train vs Test vs Validation

Split input dataset into two during [fit()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.unified_classification.UnifiedClassification.html#hana_ml.algorithms.pal.unified_classification.UnifiedClassification.fit): training (80%) and testing (the rest) using a column `Survived` and [stratified partition](https://en.wikipedia.org/wiki/Stratified_sampling).

Meaning of partitioning parameters: https://help.sap.com/docs/HANA_CLOUD_DATABASE/319d36de4fd64ac3afbf91b1fb3ce8de/b615420902974ed4a300c6ca6e7d6ab8.html

In [ ]:
uc_rdt_v2.fit(
    data=hdf_titanic_train,
    key='PassengerId', label='Survived',
    training_percent=0.8,
    partition_method='stratified', stratified_column='Survived', 
    partition_random_state=2,
    output_partition_result=True
);

### Analyze the partition of the table

Get output table names: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.pal_base.PALBase.html?highlight=get_fit_output_table_names#hana_ml.algorithms.pal.pal_base.PALBase.get_fit_output_table_names

In [ ]:
for tab in uc_rdt_v2.get_fit_output_table_names():
    print(f"Table {tab} has {myconn.table(tab).count()} record(s).")

In [ ]:
# A table with the `PARTITION_TYPE` in the name is the result of the partition
dbtable_with_partition_results=next((a_table for a_table in uc_rdt_v2.get_fit_output_table_names() if 'PARTITION_TYPE' in a_table), None)
print(f'The table that stores partition results is {dbtable_with_partition_results}')

myconn.table(dbtable_with_partition_results).collect()

In [ ]:
(myconn
    .table(dbtable_with_partition_results)
    .agg(agg_list=[("count", "TYPE", "TYPE_Count")], group_by="TYPE")
    .sort("TYPE")
    .collect()
)


In [ ]:
(myconn
    .sql(f'''SELECT TYPE, COUNT(*) AS "COUNT", ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 3) AS "DISTRIBUTION_PERCENTAGE"
    FROM {dbtable_with_partition_results}
    GROUP BY TYPE;''')
    .sort("TYPE")
    .collect()
)

## Analyze programmatically the Classifier's...

...parameters

In [ ]:
display(uc_rdt_v2.get_parameters())

Now more output tables are populated, incl. Confusion Matrix and Metrics

In [ ]:
for tab in uc_rdt_v2.model_:
    print(f"{tab.select_statement} returns {tab.count()} record(s)")

...model

In [ ]:
hdf_uc_rdt_model=uc_rdt_v2.model_[0]
display(hdf_uc_rdt_model.head(3).select(hdf_uc_rdt_model.columns[2]).collect())

...confusion matrix from the training (fitting) execution

In [ ]:
uc_rdt_v2.confusion_matrix_.collect()

🤓 **Let's discuss**:
1. Confusion Matrix
1. Statistics
1. Metrics

...statistics (see https://help.sap.com/docs/HANA_CLOUD_DATABASE/319d36de4fd64ac3afbf91b1fb3ce8de/73bbec0052b047fead450449a1454066.html?locale=en-US)

In [ ]:
uc_rdt_v2.statistics_.collect()

...metrics

In [ ]:
uc_rdt_v2.metrics_.collect()

What metrics are in the table?

In [ ]:
uc_rdt_v2.metrics_.distinct(cols='NAME').collect()

Let's check the CUMulative GAINS

In [ ]:
uc_rdt_v2.metrics_.filter("NAME = 'CUMGAINS'").sort('X').collect()

If you want to manualy plot Cumulative Gains:
```python
uc_rdt_v2.metrics_.filter("NAME = 'CUMGAINS'").sort('X').collect().plot(x='X', y='Y');
```

## Generate a model report

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(uc_rdt_v2).build().display()

🤓 **Let's discuss**:
* The Model Report

## Debrief the model

In [ ]:
from hana_ml.visualizers.model_debriefing import TreeModelDebriefing

In [ ]:
TreeModelDebriefing.tree_debrief_with_dot(uc_rdt_v2.model_[0], iframe_height=700);

🤓 **Let's discuss**:
* The Model Debriefing

# Store the model

In [ ]:
from hana_ml.model_storage import ModelStorage

Model storage: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/hana_ml.model_storage.html

In [ ]:
ms = ModelStorage(myconn)

In [ ]:
uc_rdt_v2.name = 'CodeJam-Titanic-Classification - Experiement 0x020'
ms.save_model(model=uc_rdt_v2, if_exists='replace', save_report=False)

In [ ]:
display(ms.list_models())

In [ ]:
null = None
false = False
true = True
eval(ms.list_models(name=uc_rdt_v2.name).at[0, 'JSON'])

In [ ]:
# Check what table represents your fitted model in this Python session
uc_rdt_v2.model_[0].select_statement

## Clean temp tables in your current session

In [ ]:
myconn.get_temporary_tables()

In [ ]:
# Simulate a session restart as if it would timed out or disconnected due to the communication problems
myconn.restart_session()

In [ ]:
myconn.get_temporary_tables()

# Call predition (after temp tables are cleaned)

In [ ]:
hdf_titanic_test=myconn.table('TEST', schema='TITANIC')

In [ ]:
hdf_res_v2 = uc_rdt_v2.predict(hdf_titanic_test, key = 'PassengerId')

🤓 **Let's discuss**:
1. The use of HANA temporary tables when working with PAL

# Load the saved model

In [ ]:
ms.list_models(uc_rdt_v2.name, 1)

In [ ]:
uc_rdt_v2=ms.load_model(uc_rdt_v2.name, version=1)

In [ ]:
uc_rdt_v2.model_[0].select_statement

# Call predition

In [ ]:
hdf_titanic_test=myconn.table('TEST', schema='TITANIC')

In [ ]:
hdf_res_v2 = uc_rdt_v2.predict(hdf_titanic_test, key = 'PassengerId')

In [ ]:
display(hdf_res_v2.select('PassengerId', 'SCORE', 'CONFIDENCE').sort('CONFIDENCE', desc=True).collect())

# Compare to the ground truth

In [ ]:
hdf_titanic_complete=myconn.table('COMPLETE', schema='TITANIC')

In [ ]:
hdf_res_ext=hdf_res_v2.set_index('PassengerId').join(hdf_titanic_test.set_index('PassengerId'))

In [ ]:
hdf_res_ext.head(3).collect()

In [ ]:
hdf_res_incl_groundtruth=(hdf_res_ext.set_index(['Name', 'Ticket']).join(hdf_titanic_complete.set_index(['name', 'ticket']))
                 .select('PassengerId', 'Name', 'Ticket', 'SCORE','survived',('1-ABS(SCORE-"survived")', 'IS_CORRECT'))
                 .cast('SCORE', 'INT')

)

In [ ]:
hdf_res_incl_groundtruth.head(3).collect()

In [ ]:
hdf_res_incl_groundtruth.get_table_structure()

## Accuracy

In [ ]:
hdf_res_incl_groundtruth.select(('SUM("IS_CORRECT")/COUNT("IS_CORRECT")','Accuracy')).collect()

In [ ]:
hdf_res_incl_groundtruth.agg(agg_list=[('count','PassengerId','COUNT')], group_by=['SCORE','survived']).collect()

## Confusion matrix

In [ ]:
df_res_incl_gt_pivot=(hdf_res_incl_groundtruth
 .pivot_table(values='PassengerId', index='SCORE', columns='survived', aggfunc='count')
 .collect()
);
display(df_res_incl_gt_pivot)

Order columns and rows by 0,1, and remove the Pandas DataFrame index

In [ ]:
(df_res_incl_gt_pivot[[df_res_incl_gt_pivot.columns[0]]+sorted(df_res_incl_gt_pivot.columns[1:])]
 .sort_values(by=df_res_incl_gt_pivot.columns[0], axis=0)
 .style.hide(axis='index')
)

## Using `metrics`

In [ ]:
import hana_ml.algorithms.pal.metrics as pal_metrics

Accuracy score: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.metrics.accuracy_score.html#accuracy-score

In [ ]:
pal_metrics.accuracy_score(data=hdf_res_incl_groundtruth, label_true='survived', label_pred='SCORE')

Confusion Matrix: https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2023_1_QRC/en-US/pal/algorithms/hana_ml.algorithms.pal.metrics.confusion_matrix.html#confusion-matrix

In [ ]:
hdf_cm, hdf_cr = pal_metrics.confusion_matrix(data=hdf_res_incl_groundtruth, key='PassengerId', label_true='survived', label_pred='SCORE')

In [ ]:
hdf_cm.collect()

In [ ]:
hdf_cr.collect()

In [ ]:
from hana_ml.visualizers.metrics import MetricsVisualizer
MetricsVisualizer().plot_confusion_matrix(hdf_cm, normalize=False);